# 04. Kiểm định Kỹ thuật & Lựa chọn Mô hình

In [37]:
# Cell 1
# 04. Kiểm định Kỹ thuật & Lựa chọn Mô hình

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
from clustering_library import ClusterAnalyzer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import parallel_coordinates

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Tải Dữ liệu & Khởi tạo

In [38]:
df = pd.read_csv('../data/processed/customer_features_scaled.csv')
df_scaled = df.set_index('CustomerID')
X = df.drop(columns=['CustomerID'])

## 2. Tối ưu hóa K-Means (Sử dụng lại từ 03_K-means)
Chúng tôi thực hiện tìm kiếm lưới trên các thành phần PCA và giá trị K để tìm cấu hình tối ưu.

In [39]:
# Cell 3
## 2. Tối ưu hóa K-Means (Sử dụng lại từ 03_K-means)
# Chúng tôi thực hiện tìm kiếm lưới trên các thành phần PCA và giá trị K để tìm cấu hình tối ưu.

# K_Means
# Danh sách số chiều PCA muốn thử
pca_dims = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12]

# Dải k cho K-Means
K_range = range(2, 11)

# Lưu toàn bộ kết quả
results = []
kmeans_labels_dict = {}  # Khởi tạo dictionary lưu labels

print("Tìm kiếm lưới PCA + K-Means")
print("=" * 60)

for n_comp in pca_dims:
    print(f"\nPCA with n_components = {n_comp}")
    print("-" * 40)

    # PCA
    pca = PCA(n_components=n_comp, random_state=42)
    X_pca = pca.fit_transform(X)
    explained_var = pca.explained_variance_ratio_.sum()

    inertias = []
    silhouettes = []
    dbis = []
    chs = []

    for k in K_range:
        kmeans = KMeans(
            n_clusters=k,
            random_state=42,
            n_init=10,
            max_iter=300
        )
        labels = kmeans.fit_predict(X_pca)
        
        # Lưu labels vào dictionary
        kmeans_labels_dict[(n_comp, k)] = labels

        inertias.append(kmeans.inertia_)
        silhouettes.append(silhouette_score(X_pca, labels))
        dbis.append(davies_bouldin_score(X_pca, labels))
        chs.append(calinski_harabasz_score(X_pca, labels))

        print(
            f"k={k:2d} | "
            f"Sil={silhouettes[-1]:.3f} | "
            f"DBI={dbis[-1]:.3f} | "
            f"CH={chs[-1]:.1f}"
        )

    # k tối ưu theo Silhouette
    optimal_k = K_range[np.argmax(silhouettes)]

    print(
        f"→ Optimal k (Silhouette) = {optimal_k} "
        f"| Explained variance = {explained_var:.3f}"
    )

    # Lưu kết quả
    results.append({
        "pca_dim": n_comp,
        "explained_variance": explained_var,
        "optimal_k": optimal_k,
        "best_silhouette": max(silhouettes),
        "best_dbi": min(dbis),
        "best_ch": max(chs)
    })

# Chọn cấu hình tốt nhất
if results:
    best_result = max(results, key=lambda x: x['best_silhouette'])
    best_kmeans_params = {
        'method': 'K-Means',
        'pca_dim': best_result['pca_dim'],
        'n_clusters': best_result['optimal_k'],
        'silhouette': best_result['best_silhouette'],
        'dbi': best_result['best_dbi'],
        'ch': best_result['best_ch']
    }
    
    print(f"\nBest K-Means Configuration: PCA={best_kmeans_params['pca_dim']}, k={best_kmeans_params['n_clusters']}, Silhouette={best_kmeans_params['silhouette']:.4f}")

    # Re-run to get labels
    pca = PCA(n_components=best_kmeans_params['pca_dim'], random_state=42)
    X_pca = pca.fit_transform(X)
    kmeans = KMeans(n_clusters=best_kmeans_params['n_clusters'], random_state=42, n_init=10, max_iter=300)
    best_kmeans_labels = kmeans.fit_predict(X_pca)
else:
    best_kmeans_params = {}
    best_kmeans_labels = None

Tìm kiếm lưới PCA + K-Means

PCA with n_components = 2
----------------------------------------
k= 2 | Sil=0.310 | DBI=1.233 | CH=1850.8
k= 3 | Sil=0.375 | DBI=0.912 | CH=2416.1
k= 4 | Sil=0.381 | DBI=0.865 | CH=2454.3
k= 5 | Sil=0.348 | DBI=0.902 | CH=2403.9
k= 6 | Sil=0.341 | DBI=0.950 | CH=2324.9
k= 7 | Sil=0.351 | DBI=0.882 | CH=2339.5
k= 8 | Sil=0.330 | DBI=0.915 | CH=2329.5
k= 9 | Sil=0.338 | DBI=0.870 | CH=2278.8
k=10 | Sil=0.340 | DBI=0.866 | CH=2265.4
→ Optimal k (Silhouette) = 4 | Explained variance = 0.664

PCA with n_components = 3
----------------------------------------
k= 2 | Sil=0.243 | DBI=1.523 | CH=1294.1
k= 3 | Sil=0.285 | DBI=1.226 | CH=1460.5
k= 4 | Sil=0.294 | DBI=1.139 | CH=1428.7
k= 5 | Sil=0.290 | DBI=1.098 | CH=1397.8
k= 6 | Sil=0.250 | DBI=1.112 | CH=1313.5
k= 7 | Sil=0.248 | DBI=1.185 | CH=1244.9
k= 8 | Sil=0.250 | DBI=1.162 | CH=1200.6
k= 9 | Sil=0.247 | DBI=1.125 | CH=1166.4
k=10 | Sil=0.247 | DBI=1.099 | CH=1129.7
→ Optimal k (Silhouette) = 4 | Explained

## 3. Tối ưu hóa DBSCAN 

In [40]:
# Cell 4
## 3. Tối ưu hóa DBSCAN

def select_best_eps_around_elbow(
    X,
    min_samples,
    eps_elbow,
    factors=None,
    min_cluster_points=10
):
    if eps_elbow is None:
        return None, None, []

    if factors is None:
        factors = np.linspace(0.8, 1.2, 9)

    best_row = None
    all_rows = []

    for f in factors:
        eps = float(eps_elbow * f)
        labels, n_clusters, noise_rate, sil, dbi = run_dbscan_with_metrics(
            X, eps, min_samples, min_cluster_points=min_cluster_points
        )

        row = {
            "eps": eps,
            "factor": float(f),
            "n_clusters": n_clusters,
            "noise_rate": float(noise_rate),
            "silhouette": sil,
            "dbi": dbi
        }
        all_rows.append(row)

        # chỉ xét cấu hình hợp lệ
        if sil is None or dbi is None or n_clusters < 2:
            continue

        if best_row is None:
            best_row = row
        else:
            # ưu tiên Sil cao, rồi DBI thấp, rồi noise thấp
            if (
                (row["silhouette"] > best_row["silhouette"]) or
                (row["silhouette"] == best_row["silhouette"] and row["dbi"] < best_row["dbi"]) or
                (row["silhouette"] == best_row["silhouette"] and row["dbi"] == best_row["dbi"]
                 and row["noise_rate"] < best_row["noise_rate"])
            ):
                best_row = row

    best_eps = None if best_row is None else best_row["eps"]
    return best_eps, best_row, all_rows

In [41]:
# Cell 5
def compute_eps_elbow(X, min_samples):
    """
    Tính eps tối ưu bằng k-distance elbow
    """
    n_samples = X.shape[0]
    k = max(min_samples - 1, 1)
    n_neighbors = min(k + 1, n_samples)

    if n_neighbors < 2:
        return None

    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(X)
    distances, _ = nbrs.kneighbors(X)

    k_index = min(k, distances.shape[1] - 1)
    k_distances = np.sort(distances[:, k_index])

    # Elbow hình học
    n_points = len(k_distances)
    all_points = np.vstack((np.arange(n_points), k_distances)).T

    first_point = all_points[0]
    last_point = all_points[-1]
    line_vec = last_point - first_point
    norm = np.linalg.norm(line_vec)

    if norm == 0:
        return float(k_distances[-1])

    line_vec_norm = line_vec / norm
    vec_from_first = all_points - first_point
    scalar_proj = np.dot(vec_from_first, line_vec_norm)
    proj = np.outer(scalar_proj, line_vec_norm)
    vec_to_line = vec_from_first - proj
    dist_to_line = np.linalg.norm(vec_to_line, axis=1)

    elbow_index = np.argmax(dist_to_line)
    eps_opt = float(k_distances[elbow_index])

    return eps_opt

In [42]:
# Cell 6
def run_dbscan_with_metrics(X, eps, min_samples, min_cluster_points=10):
    if eps is None:
        return None, 0, 1.0, None, None

    model = DBSCAN(eps=eps, min_samples=min_samples)
    labels = model.fit_predict(X)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_rate = float(np.mean(labels == -1))

    mask = labels != -1
    if len(np.unique(labels[mask])) > 1 and np.sum(mask) >= min_cluster_points:
        sil = float(silhouette_score(X[mask], labels[mask]))
        dbi = float(davies_bouldin_score(X[mask], labels[mask]))
    else:
        sil, dbi = None, None

    return labels, n_clusters, noise_rate, sil, dbi

In [43]:
# Cell 7
def grid_pca_dbscan(
    X_input,
    pca_dims,
    min_samples_values,
    min_cluster_points=10,
    random_state=42,
    verbose=True
):
    results = []
    labels_dict = {}
    best_row = None

    for dim in pca_dims:
        pca = PCA(n_components=dim, random_state=random_state)
        X_pca = pca.fit_transform(X_input)
        explained_var = float(pca.explained_variance_ratio_.sum())

        # lọc min_samples theo điều kiện >= dim + 1
        ms_list = [ms for ms in min_samples_values if ms >= dim + 1]

        if verbose:
            print(f"\nPCA dim={dim} | ExplainedVar={explained_var:.3f} | min_samples >= {dim+1}")
            print("-" * 70)

        for ms in ms_list:
            eps_elbow = compute_eps_elbow(X_pca, ms)

            # refine eps quanh elbow
            best_eps, best_eps_row, _ = select_best_eps_around_elbow(
                X_pca,
                min_samples=ms,
                eps_elbow=eps_elbow,
                min_cluster_points=min_cluster_points
            )

            labels, n_clusters, noise_rate, sil, dbi = run_dbscan_with_metrics(
                X_pca, best_eps, ms, min_cluster_points=min_cluster_points
            )

            labels_dict[(dim, ms)] = labels

            row = {
                "pca_dim": dim,
                "explained_variance": explained_var,
                "min_samples": ms,
                "eps_elbow": eps_elbow,
                "eps_optimal": best_eps,
                "n_clusters": n_clusters,
                "noise_rate": float(noise_rate),
                "silhouette": None if sil is None else float(sil),
                "dbi": None if dbi is None else float(dbi),
            }
            results.append(row)

            # không in dòng có sil/dbi=None
            if verbose and (sil is not None) and (dbi is not None):
                print(
                    f"✓ ms={ms:>2} | eps_elbow={eps_elbow:.5f} | eps*={best_eps:.5f} | "
                    f"clusters={n_clusters:>2} | noise={noise_rate:.1%} | Sil={sil:.3f} | DBI={dbi:.3f}"
                )

            # update best (chỉ xét cấu hình hợp lệ)
            if (sil is not None) and (dbi is not None) and (n_clusters >= 2):
                if best_row is None:
                    best_row = row
                else:
                    if (
                        (row["silhouette"] > best_row["silhouette"]) or
                        (row["silhouette"] == best_row["silhouette"] and row["dbi"] < best_row["dbi"]) or
                        (row["silhouette"] == best_row["silhouette"] and row["dbi"] == best_row["dbi"]
                         and row["noise_rate"] < best_row["noise_rate"])
                    ):
                        best_row = row

    if verbose:
        print("\nBEST CONFIG (filtered valid metrics)")
        print("=" * 70)
        if best_row is None:
            print("Không có cấu hình nào đủ điều kiện để tính Silhouette/DBI (tất cả đều None).")
        else:
            print(
                f"PCA dim={best_row['pca_dim']} | ExplainedVar={best_row['explained_variance']:.3f} | "
                f"min_samples={best_row['min_samples']} | eps_elbow={best_row['eps_elbow']:.5f} | "
                f"eps*={best_row['eps_optimal']:.5f} | clusters={best_row['n_clusters']} | "
                f"noise={best_row['noise_rate']:.1%} | Sil={best_row['silhouette']:.3f} | DBI={best_row['dbi']:.3f}"
            )

    return results, labels_dict, best_row

In [44]:
# Cell 8
pca_dims = [2, 3, 4, 5, 6, 8, 10]
min_samples_values = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

dbscan_results, dbscan_labels, best_db_row = grid_pca_dbscan(
    X_input=X,
    pca_dims=pca_dims,
    min_samples_values=min_samples_values,
    min_cluster_points=10,
    verbose=True
)

# Chọn thủ công cấu hình DBSCAN tốt nhất (PCA=3, min_samples=4 dựa trên kết quả)
manual_db_conf = (3, 4)

if manual_db_conf in dbscan_labels:
    # 1. Gán labels đúng
    best_db_labels = dbscan_labels[manual_db_conf]
    
    # 2. Cập nhật best_db_row thành dòng dữ liệu THỰC TẾ của cấu hình này
    found_row = False
    for row in dbscan_results:
        if row['pca_dim'] == 3 and row['min_samples'] == 4:
            best_db_row = row
            found_row = True
            break
            
    if found_row:
        print(f"Đã chọn thủ công DBSCAN: PCA=3, min_samples=4")
    else:
        print("Không tìm thấy metrics cho cấu hình này trong dbscan_results.")
else:
    print(f"Cấu hình {manual_db_conf} không hợp lệ (có thể do min_samples < pca_dim + 1).")
    best_db_labels = None


PCA dim=2 | ExplainedVar=0.664 | min_samples >= 3
----------------------------------------------------------------------
✓ ms= 3 | eps_elbow=0.31519 | eps*=0.37823 | clusters=20 | noise=3.4% | Sil=-0.084 | DBI=0.532
✓ ms= 4 | eps_elbow=0.35938 | eps*=0.43126 | clusters=10 | noise=3.2% | Sil=0.183 | DBI=0.531
✓ ms= 5 | eps_elbow=0.42093 | eps*=0.50512 | clusters= 6 | noise=3.1% | Sil=0.268 | DBI=0.453
✓ ms= 6 | eps_elbow=0.43972 | eps*=0.52766 | clusters= 4 | noise=3.4% | Sil=0.326 | DBI=0.445
✓ ms= 7 | eps_elbow=0.47931 | eps*=0.57517 | clusters= 3 | noise=3.4% | Sil=0.393 | DBI=0.443
✓ ms= 8 | eps_elbow=0.50227 | eps*=0.57761 | clusters= 2 | noise=4.1% | Sil=0.374 | DBI=0.476
✓ ms= 9 | eps_elbow=0.52379 | eps*=0.60235 | clusters= 2 | noise=3.9% | Sil=0.379 | DBI=0.462
✓ ms=10 | eps_elbow=0.61894 | eps*=0.58799 | clusters= 2 | noise=4.6% | Sil=0.339 | DBI=0.542
✓ ms=11 | eps_elbow=0.59351 | eps*=0.62319 | clusters= 2 | noise=4.3% | Sil=0.338 | DBI=0.553
✓ ms=12 | eps_elbow=0.69957 | e

## 4. Tối ưu hóa Phân cụm Thứ bậc

In [45]:
# Cell 9
## 4. Tối ưu hóa Phân cụm Thứ bậc

import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

def run_hierarchical_with_metrics(X, n_clusters, linkage="ward"):
    """
    Chạy Agglomerative (Hierarchical) và tính metrics trên toàn bộ điểm.

    Returns:
      labels, n_clusters, silhouette, dbi, ch
    """
    model = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    labels = model.fit_predict(X)

    # Metrics chỉ có ý nghĩa khi có >= 2 cụm
    if len(np.unique(labels)) > 1:
        sil = float(silhouette_score(X, labels))
        dbi = float(davies_bouldin_score(X, labels))
        ch  = float(calinski_harabasz_score(X, labels))
    else:
        sil, dbi, ch = None, None, None

    return labels, n_clusters, sil, dbi, ch

In [46]:
# Cell 10
from sklearn.decomposition import PCA

def grid_pca_hierarchical(
    X_input,
    pca_dims,
    k_values=range(2, 11),
    linkage_list=("ward",),
    random_state=42,
    verbose=True
):
    """
    PCA(dim) -> Hierarchical(k, linkage).
    - In best config cuối cùng (theo Sil cao nhất, tie-break DBI thấp, CH cao)
    Returns:
      results: list[dict]
      labels_dict: dict[(pca_dim, linkage, k)] = labels
      best_row: dict
    """
    results = []
    labels_dict = {}
    best_row = None

    for dim in pca_dims:
        pca = PCA(n_components=dim, random_state=random_state)
        X_pca = pca.fit_transform(X_input)
        explained_var = float(pca.explained_variance_ratio_.sum())

        for linkage in linkage_list:
            if verbose:
                print(f"\nPCA dim={dim} | ExplainedVar={explained_var:.3f} | linkage={linkage}")
                print("-" * 70)

            for k in k_values:
                labels, n_clusters, sil, dbi, ch = run_hierarchical_with_metrics(
                    X_pca, n_clusters=k, linkage=linkage
                )

                labels_dict[(dim, linkage, k)] = labels

                row = {
                    "pca_dim": dim,
                    "explained_variance": explained_var,
                    "method": "hierarchical",
                    "linkage": linkage,
                    "n_clusters": n_clusters,
                    "silhouette": sil,
                    "dbi": dbi,
                    "ch": ch
                }
                results.append(row)

                # chỉ in dòng hợp lệ
                if verbose and (sil is not None) and (dbi is not None) and (ch is not None):
                    print(f"✓ k={k:>2} | Sil={sil:.3f} | DBI={dbi:.3f} | CH={ch:.1f}")

                # update best theo: Sil ↑, DBI ↓, CH ↑
                if (sil is not None) and (dbi is not None) and (ch is not None):
                    if best_row is None:
                        best_row = row
                    else:
                        if (
                            (row["silhouette"] > best_row["silhouette"]) or
                            (row["silhouette"] == best_row["silhouette"] and row["dbi"] < best_row["dbi"]) or
                            (row["silhouette"] == best_row["silhouette"] and row["dbi"] == best_row["dbi"] and row["ch"] > best_row["ch"])
                        ):
                            best_row = row

    if verbose:
        print("\nBEST CONFIG (Hierarchical)")
        print("=" * 70)
        if best_row is None:
            print("Không có cấu hình hợp lệ để tính metric.")
        else:
            print(
                f"PCA dim={best_row['pca_dim']} | ExplainedVar={best_row['explained_variance']:.3f} | "
                f"linkage={best_row['linkage']} | k={best_row['n_clusters']} | "
                f"Sil={best_row['silhouette']:.3f} | DBI={best_row['dbi']:.3f} | CH={best_row['ch']:.1f}"
            )

    return results, labels_dict, best_row

In [47]:
# Cell 11
pca_dims = [2, 3, 4, 5, 6, 8, 10]
k_values = range(2, 11)

hier_results, hier_labels, best_hier_row = grid_pca_hierarchical(
    X_input=X,                 # hoặc X_scaled
    pca_dims=pca_dims,
    k_values=k_values,
    linkage_list=("ward", "average", "complete", "single"),
    verbose=True
)

# Chọn thủ công cấu hình tốt nhất (PCA=8, linkage=ward, k=4)
manual_conf = (8, 'ward', 4)
if manual_conf in hier_labels:
    # Gán labels theo cấu hình đã chọn
    best_hier_labels = hier_labels[manual_conf]
    
    # Cập nhật thông tin hàng (row) tốt nhất để bảng so sánh phía sau hiển thị đúng
    for row in hier_results:
        if (row['pca_dim'] == 8 and 
            row['linkage'] == 'ward' and 
            row['n_clusters'] == 4):
            best_hier_row = row
            print(f"Đã chọn thủ công Hierarchical: PCA=8, linkage=ward, k=4")
            break
else:
    print(f"Cấu hình {manual_conf} không tồn tại trong kết quả chạy thử nghiệm.")
    best_hier_labels = None


PCA dim=2 | ExplainedVar=0.664 | linkage=ward
----------------------------------------------------------------------
✓ k= 2 | Sil=0.297 | DBI=1.287 | CH=1719.2
✓ k= 3 | Sil=0.339 | DBI=0.937 | CH=2129.4
✓ k= 4 | Sil=0.333 | DBI=0.990 | CH=2175.6
✓ k= 5 | Sil=0.286 | DBI=1.000 | CH=1990.2
✓ k= 6 | Sil=0.298 | DBI=0.943 | CH=1915.3
✓ k= 7 | Sil=0.288 | DBI=1.021 | CH=1941.8
✓ k= 8 | Sil=0.284 | DBI=1.007 | CH=1992.0
✓ k= 9 | Sil=0.287 | DBI=0.966 | CH=1948.2
✓ k=10 | Sil=0.293 | DBI=0.930 | CH=1913.6

PCA dim=2 | ExplainedVar=0.664 | linkage=average
----------------------------------------------------------------------
✓ k= 2 | Sil=0.517 | DBI=0.596 | CH=230.5
✓ k= 3 | Sil=0.391 | DBI=0.576 | CH=146.9
✓ k= 4 | Sil=0.314 | DBI=0.845 | CH=366.3
✓ k= 5 | Sil=0.267 | DBI=0.721 | CH=349.9
✓ k= 6 | Sil=0.257 | DBI=0.685 | CH=285.3
✓ k= 7 | Sil=0.228 | DBI=0.694 | CH=240.9
✓ k= 8 | Sil=0.249 | DBI=0.783 | CH=575.9
✓ k= 9 | Sil=0.239 | DBI=0.787 | CH=510.1
✓ k=10 | Sil=0.237 | DBI=0.769 | CH=45

## 5. So sánh & Trực quan hóa Cuối cùng

Chúng tôi so sánh các kết quả tốt nhất từ mỗi thuật toán và trực quan hóa chúng bằng các kỹ thuật nâng cao.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_config_interactive(method, pca_dim, param1, linkage=None):
    """Visualize clustering for selected configuration with interactive controls"""
    
    with output_plot:
        clear_output(wait=True)
        
        print(f"Visualizing {method} with:")
        print(f"  - PCA Dimensions: {pca_dim}")
        print(f"  - Parameter: {param1}")
        if linkage:
            print(f"  - Linkage: {linkage}")
        print("-" * 40)
        
        try:
            # Get labels based on method
            labels = None
            
            if method == 'K-Means':
                if 'kmeans_labels_dict' in globals() and (pca_dim, param1) in kmeans_labels_dict:
                    labels = kmeans_labels_dict[(pca_dim, param1)]
                    title = f'K-Means (PCA={pca_dim}, k={param1})'
                else:
                    print(f"⚠️ Configuration not found for K-Means")
                    return
                    
            elif method == 'DBSCAN':
                if 'dbscan_labels' in globals() and (pca_dim, param1) in dbscan_labels:
                    labels = dbscan_labels[(pca_dim, param1)]
                    title = f'DBSCAN (PCA={pca_dim}, min_samples={param1})'
                else:
                    print(f"⚠️ Configuration not found for DBSCAN")
                    return
                    
            elif method == 'Hierarchical':
                if linkage and 'hier_labels' in globals() and (pca_dim, linkage, param1) in hier_labels:
                    labels = hier_labels[(pca_dim, linkage, param1)]
                    title = f'Hierarchical (PCA={pca_dim}, k={param1}, linkage={linkage})'
                else:
                    # Try to find any linkage
                    if 'hier_labels' in globals():
                        for link in ['ward', 'average', 'complete', 'single']:
                            if (pca_dim, link, param1) in hier_labels:
                                labels = hier_labels[(pca_dim, link, param1)]
                                title = f'Hierarchical (PCA={pca_dim}, k={param1}, linkage={linkage if linkage else link})'
                                break
                    if labels is None:
                        print(f"⚠️ Configuration not found for Hierarchical")
                        return
            
            if labels is None:
                print("❌ No labels found")
                return
            
            # Calculate metrics
            if len(np.unique(labels)) > 1 and -1 not in labels:
                sil = silhouette_score(X, labels)
                dbi = davies_bouldin_score(X, labels)
                print(f"📊 Metrics: Silhouette={sil:.3f}, DBI={dbi:.3f}")
            elif len(np.unique(labels)) > 1:
                # Remove noise for metrics calculation
                mask = labels != -1
                if len(np.unique(labels[mask])) > 1:
                    sil = silhouette_score(X[mask], labels[mask])
                    dbi = davies_bouldin_score(X[mask], labels[mask])
                    print(f"📊 Metrics (noise excluded): Silhouette={sil:.3f}, DBI={dbi:.3f}")
            
            # Cluster statistics
            unique, counts = np.unique(labels, return_counts=True)
            print(f"📈 Cluster sizes:")
            for label, count in zip(unique, counts):
                if label == -1:
                    print(f"  Noise: {count} samples ({count/len(labels)*100:.1f}%)")
                else:
                    print(f"  Cluster {label}: {count} samples ({count/len(labels)*100:.1f}%)")
            
            # PCA for visualization (2D and 3D)
            pca_2d = PCA(n_components=2, random_state=42)
            X_2d = pca_2d.fit_transform(X)
            
            pca_3d = PCA(n_components=3, random_state=42)
            X_3d = pca_3d.fit_transform(X)
            
            # Create subplots (2x2 grid)
            fig = plt.figure(figsize=(20, 12))
            
            # Plot 1: PCA 2D
            ax1 = fig.add_subplot(2, 2, 1)
            if -1 in labels:
                noise_mask = labels == -1
                cluster_mask = ~noise_mask
                
                scatter1 = ax1.scatter(X_2d[cluster_mask, 0], X_2d[cluster_mask, 1], 
                                     c=labels[cluster_mask], cmap='viridis', 
                                     s=30, alpha=0.8, label='Clusters')
                ax1.scatter(X_2d[noise_mask, 0], X_2d[noise_mask, 1], 
                          c='black', marker='x', s=50, label='Noise')
                ax1.legend()
            else:
                scatter1 = ax1.scatter(X_2d[:, 0], X_2d[:, 1], 
                                     c=labels, cmap='viridis', s=30, alpha=0.8)
                plt.colorbar(scatter1, ax=ax1, label='Cluster')
            
            ax1.set_title(f'{title}\n(PCA 2D - {pca_2d.explained_variance_ratio_[:2].sum():.1%} variance)')
            ax1.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
            ax1.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
            ax1.grid(True, alpha=0.3)
            
            # Plot 2: PCA 3D
            ax2 = fig.add_subplot(2, 2, 2, projection='3d')
            if -1 in labels:
                noise_mask = labels == -1
                cluster_mask = ~noise_mask
                
                scatter2 = ax2.scatter(X_3d[cluster_mask, 0], X_3d[cluster_mask, 1], X_3d[cluster_mask, 2],
                                     c=labels[cluster_mask], cmap='viridis', 
                                     s=30, alpha=0.7, label='Clusters')
                ax2.scatter(X_3d[noise_mask, 0], X_3d[noise_mask, 1], X_3d[noise_mask, 2],
                          c='black', marker='x', s=50, label='Noise')
                ax2.legend()
            else:
                scatter2 = ax2.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2],
                                     c=labels, cmap='viridis', s=30, alpha=0.7)
            
            ax2.set_title(f'PCA 3D\n({pca_3d.explained_variance_ratio_[:3].sum():.1%} variance)')
            ax2.set_xlabel(f'PC1 ({pca_3d.explained_variance_ratio_[0]:.1%})')
            ax2.set_ylabel(f'PC2 ({pca_3d.explained_variance_ratio_[1]:.1%})')
            ax2.set_zlabel(f'PC3 ({pca_3d.explained_variance_ratio_[2]:.1%})')
            
            # Plot 3: t-SNE
            ax3 = fig.add_subplot(2, 2, 3)
            print("  Running t-SNE (this may take a moment)...")
            tsne = TSNE(n_components=2, random_state=42, perplexity=30)
            X_tsne = tsne.fit_transform(X)
            
            if -1 in labels:
                noise_mask = labels == -1
                cluster_mask = ~noise_mask
                
                scatter3 = ax3.scatter(X_tsne[cluster_mask, 0], X_tsne[cluster_mask, 1], 
                                     c=labels[cluster_mask], cmap='viridis', 
                                     s=30, alpha=0.8, label='Clusters')
                ax3.scatter(X_tsne[noise_mask, 0], X_tsne[noise_mask, 1], 
                          c='black', marker='x', s=50, label='Noise')
                ax3.legend()
            else:
                scatter3 = ax3.scatter(X_tsne[:, 0], X_tsne[:, 1], 
                                     c=labels, cmap='viridis', s=30, alpha=0.8)
                plt.colorbar(scatter3, ax=ax3, label='Cluster')
            
            ax3.set_title('t-SNE Visualization')
            ax3.set_xlabel('t-SNE 1')
            ax3.set_ylabel('t-SNE 2')
            ax3.grid(True, alpha=0.3)
            
            # Plot 4: Cluster distribution
            ax4 = fig.add_subplot(2, 2, 4)
            unique_labels_sorted = sorted(unique)
            colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels_sorted)))
            
            for i, label in enumerate(unique_labels_sorted):
                if label == -1:
                    color = 'black'
                    label_name = 'Noise'
                else:
                    color = colors[i]
                    label_name = f'Cluster {label}'
                
                count = counts[list(unique).index(label)]
                ax4.bar(label_name, count, color=color, alpha=0.8)
                ax4.text(i, count + max(counts)*0.01, str(count), 
                        ha='center', va='bottom', fontsize=10)
            
            ax4.set_title('Cluster Distribution')
            ax4.set_ylabel('Number of Samples')
            ax4.tick_params(axis='x', rotation=45)
            ax4.grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"❌ Error in visualization: {e}")
            import traceback
            traceback.print_exc()

method_dropdown = widgets.Dropdown(
    options=['K-Means', 'DBSCAN', 'Hierarchical'],
    value='K-Means',
    description='Algorithm:',
    style={'description_width': 'initial'}
)

pca_slider = widgets.IntSlider(
    continuous_update=False,
    value=3,
    min=2,
    max=min(10, X.shape[1]),
    step=1,
    description='PCA Dims:',
    style={'description_width': 'initial'}
)

# Dynamic parameter widgets
param1_slider = widgets.IntSlider(
    continuous_update=False, 
    description='k (Clusters):', 
    min=2, 
    max=15, 
    value=4, 
    style={'description_width': 'initial'}
)

linkage_dropdown = widgets.Dropdown(
    options=['ward', 'average', 'complete', 'single'],
    value='ward',
    description='Linkage:',
    style={'description_width': 'initial'},
    disabled=False
)

output_plot = widgets.Output()
output_message = widgets.Output()

def update_params(*args):
    method = method_dropdown.value
    
    if method == 'K-Means':
        param1_slider.description = 'k (Clusters):'
        param1_slider.min, param1_slider.max, param1_slider.step = 2, 15, 1
        param1_slider.value = 4
        param1_slider.layout.display = 'flex'
        linkage_dropdown.layout.display = 'none'
        
    elif method == 'DBSCAN':
        param1_slider.description = 'Min Samples:'
        param1_slider.min, param1_slider.max, param1_slider.step = 2, 20, 1
        param1_slider.value = 4
        param1_slider.layout.display = 'flex'
        linkage_dropdown.layout.display = 'none'
        
    elif method == 'Hierarchical':
        param1_slider.description = 'k (Clusters):'
        param1_slider.min, param1_slider.max, param1_slider.step = 2, 15, 1
        param1_slider.value = 4
        param1_slider.layout.display = 'flex'
        linkage_dropdown.layout.display = 'flex'

method_dropdown.observe(update_params, 'value')

def save_config(b):
    with output_message:
        clear_output()
        method = method_dropdown.value
        pca_dim = pca_slider.value
        p1 = param1_slider.value
        linkage = linkage_dropdown.value if method == 'Hierarchical' else None
        
        print(f"Saving configuration for {method}...")
        
        selected_labels = None
        col_name = ""
        
        if method == 'K-Means':
            col_name = 'Cluster_KMeans'
            if 'kmeans_labels_dict' in globals() and (pca_dim, p1) in kmeans_labels_dict:
                selected_labels = kmeans_labels_dict[(pca_dim, p1)]
            else:
                print(f"❌ Error: Configuration (PCA={pca_dim}, k={p1}) not found for K-Means.")
                return
                
        elif method == 'DBSCAN':
            col_name = 'Cluster_DBSCAN'
            if 'dbscan_labels' in globals() and (pca_dim, p1) in dbscan_labels:
                selected_labels = dbscan_labels[(pca_dim, p1)]
            else:
                print(f"❌ Error: Configuration (PCA={pca_dim}, MinSamp={p1}) not found for DBSCAN.")
                return

        elif method == 'Hierarchical':
            col_name = 'Cluster_Hierarchical'
            if 'hier_labels' in globals() and (pca_dim, linkage, p1) in hier_labels:
                selected_labels = hier_labels[(pca_dim, linkage, p1)]
            else:
                print(f"❌ Error: Configuration (PCA={pca_dim}, linkage={linkage}, k={p1}) not found.")
                return

        if selected_labels is not None:
            try:
                # Load existing or create new
                csv_path = "../data/processed/clustered_customers.csv"
                try:
                    save_df = pd.read_csv(csv_path, index_col=0)
                    # Align indices with current analysis data to fix any length mismatches
                    save_df = save_df.reindex(df_scaled.index)
                except FileNotFoundError:
                    save_df = pd.DataFrame(index=df_scaled.index)
                
                # Ensure length match
                # Assign labels (safe now as indices are aligned)
                save_df[col_name] = pd.Series(selected_labels, index=df_scaled.index)
                
                save_df.to_csv(csv_path)
                print(f"✅ Successfully saved {method} labels to '{col_name}' in clustered_customers.csv")
                print(f"   Configuration: PCA={pca_dim}, " + 
                      (f"linkage={linkage}, " if linkage else "") + 
                      f"param={p1}")
                
            except Exception as e:
                print(f"❌ Error saving to CSV: {e}")
        else:
            print("❌ No labels to save.")

def plot_event(b=None):
    method = method_dropdown.value
    pca_dim = pca_slider.value
    p1 = param1_slider.value
    linkage = linkage_dropdown.value if method == 'Hierarchical' else None
    
    plot_config_interactive(method, pca_dim, p1, linkage)

save_btn = widgets.Button(
    description='Save Configuration',
    button_style='success',
    tooltip='Save current specific algorithm results to CSV',
    icon='save'
)
save_btn.on_click(save_config)

visualize_btn = widgets.Button(
    description='Visualize',
    button_style='info',
    tooltip='Update visualization with current parameters',
    icon='refresh'
)
visualize_btn.on_click(plot_event)

# ==================== LAYOUT ====================
ui = widgets.VBox([
    widgets.HBox([
        method_dropdown, 
        widgets.VBox([pca_slider, linkage_dropdown])
    ]),
    widgets.HBox([
        param1_slider,
        visualize_btn,
        save_btn
    ]),
    output_message,
    output_plot
])

update_params()
print("Interactive Clustering Visualization Ready!")
print("Select algorithm and parameters, then click 'Visualize'")
print("✨ Now with 3D PCA visualization!")
print("-" * 50)
display(ui)

plot_config_interactive('K-Means', 3, 4, None)

Interactive Clustering Visualization Ready!
Select algorithm and parameters, then click 'Visualize'
✨ Now with 3D PCA visualization!
--------------------------------------------------
